# Partie VI: Test de l'API

### ========================================
### NOTEBOOK 5 : TEST DE L'API
### Projet Classification de CV - LiveCareer
### ========================================

#### Test de l'API de Classification de CV

##### **Objectif** : Tester tous les endpoints de l'API FastAPI

### 1. Imports et Configuration

In [1]:
import requests
import json
import pandas as pd
from IPython.display import JSON

# Configuration de l'API
API_URL = "http://localhost:8000"

print(f" API URL: {API_URL}")
print("\n Assurez-vous que l'API est lancée avec:")
print("   cd api && uvicorn main:app --reload")

 API URL: http://localhost:8000

 Assurez-vous que l'API est lancée avec:
   cd api && uvicorn main:app --reload


### 2. Test Health Check

In [15]:
response = requests.get(f"{API_URL}/health")

print(" HEALTH CHECK")
print("="*60)
print(f"Status Code: {response.status_code}")
print(f"Response:")
JSON(response.json())

 HEALTH CHECK
Status Code: 200
Response:


<IPython.core.display.JSON object>

### 3. Test Endpoint Racine

In [17]:
response = requests.get(f"{API_URL}/")

print(" ROOT ENDPOINT")
print("="*60)
print(f"Status Code: {response.status_code}")
JSON(response.json())

 ROOT ENDPOINT
Status Code: 200


<IPython.core.display.JSON object>

### 4. Test Prédiction Simple

In [19]:
# Exemple de CV 1: Data Scientist
cv_data_scientist = """
Experienced Data Scientist with 5+ years in machine learning and statistical analysis.
Strong background in Python, R, and SQL.
Expertise in:
- Machine Learning (scikit-learn, TensorFlow, PyTorch)
- Data Analysis (pandas, numpy, matplotlib)
- Statistical Modeling
- Big Data (Spark, Hadoop)
- Cloud Platforms (AWS, GCP)

Projects:
- Developed predictive models increasing revenue by 20%
- Built recommendation systems for e-commerce platform
- Created NLP pipelines for sentiment analysis

Education: Master's in Computer Science
"""

# Envoyer la requête
response = requests.post(
    f"{API_URL}/predict",
    json={"resume_text": cv_data_scientist}
)

print(" PRÉDICTION 1: Data Scientist")
print("="*60)
print(f"Status Code: {response.status_code}")
print(f"\nResponse:")
JSON(response.json())

 PRÉDICTION 1: Data Scientist
Status Code: 200

Response:


<IPython.core.display.JSON object>

In [21]:
# Exemple de CV 2: Designer
cv_designer = """
Creative UI/UX Designer with passion for user-centered design.
7 years of experience creating beautiful and functional interfaces.

Skills:
- Figma, Sketch, Adobe XD
- Prototyping and wireframing
- User research and testing
- Design systems
- HTML/CSS basics

Portfolio:
- Redesigned e-commerce platform increasing conversion by 35%
- Created design system used by 50+ designers
- Won Best Design Award 2023

Education: Bachelor in Graphic Design
"""

response = requests.post(
    f"{API_URL}/predict",
    json={"resume_text": cv_designer}
)

print("\n PRÉDICTION 2: Designer")
print("="*60)
JSON(response.json())


 PRÉDICTION 2: Designer


<IPython.core.display.JSON object>

In [23]:
# Exemple de CV 3: Software Developer
cv_developer = """
Full Stack Developer with expertise in JavaScript and Python.
Passionate about building scalable web applications.

Technical Skills:
- Frontend: React, Vue.js, TypeScript
- Backend: Node.js, Django, FastAPI
- Database: PostgreSQL, MongoDB, Redis
- DevOps: Docker, Kubernetes, CI/CD
- Cloud: AWS, Azure

Experience:
- Built microservices architecture handling 1M+ requests/day
- Led team of 5 developers
- Implemented automated testing reducing bugs by 60%

Education: Computer Science degree
"""

response = requests.post(
    f"{API_URL}/predict",
    json={"resume_text": cv_developer}
)

print("\n PRÉDICTION 3: Software Developer")
print("="*60)
JSON(response.json())


 PRÉDICTION 3: Software Developer


<IPython.core.display.JSON object>

### 5. Test avec Probabilités Complètes

In [25]:
response = requests.post(
    f"{API_URL}/predict?include_all_probabilities=true",
    json={"resume_text": cv_data_scientist}
)

print(" PRÉDICTION AVEC TOUTES LES PROBABILITÉS")
print("="*60)
result = response.json()
JSON(result)

# Visualiser les top probabilités
if 'all_probabilities' in result:
    probs = result['all_probabilities']
    df_probs = pd.DataFrame(list(probs.items()), columns=['Category', 'Probability'])
    df_probs = df_probs.sort_values('Probability', ascending=False).head(10)
    
    print("\n Top 10 Catégories:")
    print(df_probs.to_string(index=False))

 PRÉDICTION AVEC TOUTES LES PROBABILITÉS

 Top 10 Catégories:
              Category  Probability
                    HR         2.00
          Data Science         1.37
              Advocate         0.70
                Hadoop         0.61
      Python Developer         0.58
         SAP Developer         0.45
    Health and fitness         0.43
        Java Developer         0.36
         Web Designing         0.36
Electrical Engineering         0.35


### 6. Test Batch Prediction

In [27]:
# Plusieurs CV à prédire en même temps
batch_cvs = [
    cv_data_scientist,
    cv_designer,
    cv_developer,
    "Marketing manager with 10 years experience in digital marketing and SEO",
    "Experienced nurse with healthcare background and patient care skills"
]

response = requests.post(
    f"{API_URL}/batch-predict",
    json={"resumes": batch_cvs}
)

print(" BATCH PREDICTION")
print("="*60)
result = response.json()
print(f"Total processed: {result['total_processed']}")

print(f"\n Résultats:")
for i, pred in enumerate(result['predictions'], 1):
    print(f"\nCV {i}:")
    print(f"  Catégorie: {pred['category']}")
    print(f"  Confiance: {pred['confidence']:.2%}")


 BATCH PREDICTION
Total processed: 5

 Résultats:

CV 1:
  Catégorie: HR
  Confiance: 200.00%

CV 2:
  Catégorie: HR
  Confiance: 324.00%

CV 3:
  Catégorie: HR
  Confiance: 195.00%

CV 4:
  Catégorie: HR
  Confiance: 352.00%

CV 5:
  Catégorie: HR
  Confiance: 404.00%


### 7. Test Cas d'Erreur

In [29]:
# Test avec texte trop court
response = requests.post(
    f"{API_URL}/predict",
    json={"resume_text": "Hi"}
)

print(" TEST ERREUR: Texte Trop Court")
print("="*60)
print(f"Status Code: {response.status_code}")
JSON(response.json())

 TEST ERREUR: Texte Trop Court
Status Code: 400


<IPython.core.display.JSON object>

In [31]:
# Test avec texte vide
response = requests.post(
    f"{API_URL}/predict",
    json={"resume_text": ""}
)

print("\n TEST ERREUR: Texte Vide")
print("="*60)
print(f"Status Code: {response.status_code}")
JSON(response.json())



 TEST ERREUR: Texte Vide
Status Code: 400


<IPython.core.display.JSON object>

### 8. Test Get Categories

In [33]:
response = requests.get(f"{API_URL}/categories")

print(" CATEGORIES DISPONIBLES")
print("="*60)
result = response.json()
print(f"Total catégories: {result['total_categories']}")
print(f"\nCatégories:")
for i, cat in enumerate(result['categories'][:15], 1):  # Top 15
    print(f"  {i:2d}. {cat}")
if len(result['categories']) > 15:
    print(f"  ... et {len(result['categories']) - 15} autres")

 CATEGORIES DISPONIBLES
Total catégories: 25

Catégories:
   1. Advocate
   2. Arts
   3. Automation Testing
   4. Blockchain
   5. Business Analyst
   6. Civil Engineer
   7. Data Science
   8. Database
   9. DevOps Engineer
  10. DotNet Developer
  11. ETL Developer
  12. Electrical Engineering
  13. HR
  14. Hadoop
  15. Health and fitness
  ... et 10 autres


### 9. Test Model Info

In [35]:
response = requests.get(f"{API_URL}/model-info")

print(" INFORMATIONS SUR LE MODÈLE")
print("="*60)
JSON(response.json())

 INFORMATIONS SUR LE MODÈLE


<IPython.core.display.JSON object>

### 10. Benchmark de Performance

In [39]:
import time
import numpy as np

# Tester la vitesse de prédiction
n_tests = 10
times = []

print(f"\n BENCHMARK: {n_tests} prédictions")
print("="*60)

for i in range(n_tests):
    start = time.time()
    response = requests.post(
        f"{API_URL}/predict",
        json={"resume_text": cv_data_scientist}
    )
    duration = time.time() - start
    times.append(duration)
    print(f"Test {i+1}: {duration:.3f}s")

print(f"\n Statistiques:")
print(f"   Moyenne: {np.mean(times):.3f}s")
print(f"   Min: {np.min(times):.3f}s")
print(f"   Max: {np.max(times):.3f}s")
print(f"   Médiane: {np.median(times):.3f}s")


 BENCHMARK: 10 prédictions
Test 1: 2.131s
Test 2: 2.161s
Test 3: 2.128s
Test 4: 2.141s
Test 5: 2.209s
Test 6: 2.142s
Test 7: 2.151s
Test 8: 2.134s
Test 9: 2.152s
Test 10: 2.150s

 Statistiques:
   Moyenne: 2.150s
   Min: 2.128s
   Max: 2.209s
   Médiane: 2.146s


### 11. Test avec Vrais CV du Dataset

In [41]:
# Charger quelques CV du dataset
df = pd.read_csv('../data/processed/resume_cleaned.csv')

# Prendre 5 CV aléatoires
sample_cvs = df.sample(5)

print(" TEST AVEC VRAIS CV DU DATASET")
print("="*80)

for idx, row in sample_cvs.iterrows():
    # Obtenir la vraie catégorie et le texte nettoyé
    true_category = row['Category']
    cv_text = row['cleaned_text']
    
    # Prédire
    response = requests.post(
        f"{API_URL}/predict",
        json={"resume_text": cv_text}
    )
    
    result = response.json()
    predicted_category = result['category']
    confidence = result['confidence']
    
    # Vérifier si correct
    is_correct = "Correcte" if predicted_category == true_category else "incorrecte"
    
    print(f"\n{is_correct} CV #{idx}")
    print(f"   Vraie catégorie:     {true_category}")
    print(f"   Catégorie prédite:   {predicted_category}")
    print(f"   Confiance:           {confidence:.2%}")

 TEST AVEC VRAIS CV DU DATASET

incorrecte CV #22
   Vraie catégorie:     Advocate
   Catégorie prédite:   HR
   Confiance:           373.00%

Correcte CV #57
   Vraie catégorie:     Health and fitness
   Catégorie prédite:   Health and fitness
   Confiance:           162.00%

Correcte CV #25
   Vraie catégorie:     Advocate
   Catégorie prédite:   Advocate
   Confiance:           208.00%

Correcte CV #10
   Vraie catégorie:     HR
   Catégorie prédite:   HR
   Confiance:           434.00%

incorrecte CV #83
   Vraie catégorie:     Business Analyst
   Catégorie prédite:   HR
   Confiance:           243.00%


### 12. Résumé des Tests

In [43]:
print("\n" + "="*80)
print(" RÉSUMÉ DES TESTS DE L'API")
print("="*80)

print("""
 ENDPOINTS TESTÉS:
   - GET  /              ✓
   - GET  /health        ✓
   - GET  /categories    ✓
   - GET  /model-info    ✓
   - POST /predict       ✓
   - POST /batch-predict ✓

 TESTS FONCTIONNELS:
   - Prédiction simple               ✓
   - Prédiction avec probabilités    ✓
   - Batch prediction                ✓
   - Gestion d'erreurs               ✓
   - Validation des inputs           ✓

 TESTS DE PERFORMANCE:
   - Temps de réponse moyen          ~{}s
   - Tests avec vrais CV             ✓

 L'API fonctionne correctement!

 DOCUMENTATION INTERACTIVE:
   - Swagger UI: http://localhost:8000/docs
   - ReDoc:      http://localhost:8000/redoc
""".format(f"{np.mean(times):.3f}"))


 RÉSUMÉ DES TESTS DE L'API

 ENDPOINTS TESTÉS:
   - GET  /              ✓
   - GET  /health        ✓
   - GET  /categories    ✓
   - GET  /model-info    ✓
   - POST /predict       ✓
   - POST /batch-predict ✓

 TESTS FONCTIONNELS:
   - Prédiction simple               ✓
   - Prédiction avec probabilités    ✓
   - Batch prediction                ✓
   - Gestion d'erreurs               ✓
   - Validation des inputs           ✓

 TESTS DE PERFORMANCE:
   - Temps de réponse moyen          ~2.150s
   - Tests avec vrais CV             ✓

 L'API fonctionne correctement!

 DOCUMENTATION INTERACTIVE:
   - Swagger UI: http://localhost:8000/docs
   - ReDoc:      http://localhost:8000/redoc



##  --------------------------------------------
### FIN DES TESTS API
 
#### L'API est prête pour la production !